# Сложные объединения

## 1. Знакомимся с данными


Таблица city — это справочник городов. Структура справочника представлена ниже.
Название поля 	Тип данных 	Описание
city_id 	    integer 	уникальный идентификатор города, первичный ключ
city_name   	text 	название города
state 	        text 	штат, к которому относится город
population 	    integer 	население города
area 	        numeric 	площадь города

Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:
Название поля 	Тип данных 	Описание
cust_id 	    integer 	уникальный идентификатор клиента, первичный ключ
cust_name   	text 	    название клиента
annual_revenue 	numeric 	ежегодная выручка
cust_type   	text    	тип пользователя
address 	    text 	    адрес
zip 	        integer 	почтовый индекс
phone       	text    	телефон
city_id        	integer 	идентификатор города, внешний ключ к таблице city

Следующая таблица — driver — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.
Название поля 	Тип данных 	Описание
driver_id 	integer 	уникальный идентификатор водителя, первичный ключ
first_name 	text 	    имя водителя
last_name 	text 	    фамилия водителя
address 	text 	    адрес водителя
zip_code 	integer 	почтовый индекс водителя
phone 	    text 	    телефон водителя
city_id 	integer 	идентификатор города водителя, внешний ключ к таблице city

В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:
Название поля 	Тип данных 	Описание
truck_id 	integer 	Уникальный идентификатор грузовика, первичный ключ
make 	    text 	    Производитель грузовика
model_year 	integer 	Дата выпуска грузовика

Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.
Название поля 	Тип данных 	Описание
ship_id 	integer 	уникальный идентификатор доставки, первичный ключ
cust_id 	integer 	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer
weight 	    numeric 	вес посылки
truck_id 	integer 	идентификатор грузовика, на котором отправлена доставка, внешний ключк таблице truck
driver_id 	integer 	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver
city_id 	integer 	идентификатор города в который совершена доставка, внешний ключ к таблице city
ship_date 	date 	    дата доставки

# 2. UNION
## Принцип и условия работы Union
Для этого напишем простой запрос:

SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name, задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/
FROM public.books /*из схемы public и таблицы books*/
UNION ALL /*оператор присоединения*/
SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/
FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:
одинаковый тип данных;
одинаковое количество столбцов;
одинаковый порядок столбцов согласно типу данных.

### Виды UNION

Оператор присоединения существует в двух вариантах:

        UNION выводит только уникальные записи;
        UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.

### Синтаксис

Запрос строится таким образом:

SELECT
         n columns
FROM 
         table_1

UNION ALL

SELECT 
         n columns
FROM 
         table_2
...
UNION ALL

SELECT 
         n columns
FROM 
         table_n

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.
Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

SELECT
         c.city_id object_name, 'id города' object_type
FROM 
         sql.city c
UNION ALL

SELECT
         d.driver_id other_name, 'id водителя' other_type
FROM 
         sql.driver d
UNION ALL

SELECT
         s.ship_id, 'id доставки'
FROM 
         sql.shipment s
UNION ALL

SELECT
         c.cust_id, 'id клиента'
FROM 
         sql.customer c
UNION ALL

SELECT
         t.truck_id, 'id грузовика'
FROM 
         sql.truck t
ORDER BY 1

Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.

В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.

Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!

Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
Выдается ошибка!

Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)

___
Напишите запрос, который создаёт уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков.
Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck).
Отсортируйте список по названию объекта, а затем — по типу.

SELECT c.city_name "название", 'city' "тип объекта"
FROM sql.city c
UNION 
SELECT c.state, 'state' 
FROM sql.city c
UNION 
SELECT d.first_name, 'driver' 
FROM  sql.driver d
UNION 
select t.make, 'truck'
 from sql.truck t
 order by 1,2

___
Напишите запрос, который соберёт имена всех упомянутых городов и штатов с таблицы city.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT c.city_name object_name
FROM sql.city c
UNION all
SELECT c.state
FROM sql.city c
order by 1

___
Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов.
Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

SELECT c.city_name object_name
FROM sql.city c
UNION all
SELECT c.state
FROM sql.city c
order by 1

# 3. UNION и ограничение типов данных

## Почему так важен тип данных?